# Feature engineering

In [27]:
# Traitement de données
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy.stats import spearmanr
from sklearn.model_selection import train_test_split

In [28]:
X = pd.read_csv("X_train_NHkHMNU.csv")
y = pd.read_csv("y_train_ZAN5mwg.csv")

df = pd.concat([X, y], axis=1)

df = df.drop(df.columns[-2], axis=1)

Change Nan Values

In [29]:
numeric_cols = df.select_dtypes(include=["number"]).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

### Séparation des dataframes fr et de

In [30]:
df_fr = df[df["COUNTRY"] == "FR"]
df_de = df[df["COUNTRY"] == "DE"]

##### Seuils pour df_fr
- COAL_RET < 0.8
- FR_CONSUMPTION > 1.5
- FR_NUCLEAR < -1.8
- FR_HYDRO < -0.4

##### Seuils pour df_de
- DE_NET_IMPORT < 0.45
- DE_CONSUMPTION > 1.2
- DE_NET_EXPORT > -0.45
- DE_WINDPOW > 0.3


Transformation "ReLU"

In [31]:
from typing import List

def AddSeuilColumn(df: pd.DataFrame, column_name: str, seuil: float, way: str):
    message = column_name + "_Threshold_" + str(seuil)
    if way == "sup":
        df[message] = df[column_name].where(df[column_name] >= seuil, 0)
    else:
        df[message] = df[column_name].where(df[column_name] <= seuil, 0)

#### Colonnes à garder df_fr:
- COAL_RET (seuil)
- FR_CONSUMPTION (seuil)
- FR_NUCLEAR (seuil)
- FR_HYDRO (seuil)

Parceque coeff de spearman avec la Target > 0.07

- DE_NET_EXPORT   
- DE_NET_IMPORT    
- DE_HYDRO         
- DE_WINDPOW      
- FR_WINDPOW      
- GAS_RET          
- CARBON_RET       

ça serait bien de réduire le nombre de colonne non-seuil en faisant une pca (et de regrouper variables PCA et seuil après)   

#### Colonnes à garder df_de

- DE_NET_IMPORT (seuil)
- DE_CONSUMPTION (seuil)
- DE_NET_EXPORT (seuil)
- DE_WINDPOW (seuil)

Parceque coeff de spearman avec la Target > 0.10

- DE_GAS              
- DE_COAL             
- DE_HYDRO            
- FR_WINDPOW         
- DE_LIGNITE          
- DE_RESIDUAL_LOAD    
- DE_WIND            -


ça serait bien de réduire le nombre de colonne non-seuil en faisant une pca (et de regrouper variables PCA et seuil après) 

## Modèle de base

In [32]:
X_all = df.drop(columns=["TARGET", "COUNTRY"])
y_all = df["TARGET"]


X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2, random_state=42)

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred_train = lr.predict(X_train)
y_pred_test  = lr.predict(X_test)

def spearman_corr(y_true, y_pred):
    return spearmanr(y_true, y_pred).correlation

print("Corrélation (Spearman) train : {:.1f}%".format(100 * spearman_corr(y_train, y_pred_train)))
print("Corrélation (Spearman) test  : {:.1f}%".format(100 * spearman_corr(y_test,  y_pred_test)))

Corrélation (Spearman) train : 29.2%
Corrélation (Spearman) test  : 20.2%


#### Modèle sur df_fr (En utilant le df contenant les colonnes from PCA plus colonnes seuils)

Et regression pour le moment


#

#### Modèle sur df_de (En utilant le de contenant les colonnes from PCA plus colonnes seuils)

Regression aussi

#### Assemblage des deux modèles et train/test comme le modèle de base pour comparer 
Il faudra penser, si on a le temps à faire du k-fold pour éviter l'overfitting, genre on divise en 5 morceau et à chaques fois on change les morceaux qui entrainent et qui test et on voit si le modèle généralise bien.

#### Autres modèles à faire ensuite

### Polynomiale Regression (vue lab2)

### Decision Tree Regressor (vue lab2)